In [64]:
# importing module
from pandas import *
from statistics import mean
import numpy as np
from helper760 import read_inputs
import tensorflow as tf

test_case = 0

In [65]:
Clininc_Data,Outcome_Data,CT_Data = read_inputs()

# Test Case 0 - Keras CT-To-Cancer Prediction
This is a test run perform prediction of cancer using CT Data. This test run uses a neural network constructed using Keras

# Test Case 1 - NN CT+Clinic-To-Cancer Prediction
In this test run we tried to use CT_Data + Clinic Data to predict cancer, Use Knn

In [66]:
if test_case == 0:
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import KFold
    n_split = 5

    param_grid = [
        {
            'weights': ['uniform'], 
            'n_neighbors': [i for i in range(1, 10)]
        },
        {
            'weights': ['distance'],
            'n_neighbors': [i for i in range(1, 10)], 
            'p': [i for i in range(1, 10)]
        }
    ]



    X = np.array(CT_Data)
    X = X.T
    X = X.astype(float)
    X_normed = (X - X.min(0)) / X.ptp(0)
    y = np.array(Outcome_Data[21])
    y = y.astype(int)
    
    counter = 0
    for train_index,test_index in KFold(n_split).split(X):

        print("fold "+str(counter))
        counter = counter + 1

        x_train,x_test = X_normed[train_index,:], X_normed[test_index,:]
        y_train,y_test = y[train_index], y[test_index]


        knn_clf = KNeighborsClassifier()
        grid_search = GridSearchCV(knn_clf, param_grid , cv = 1, scoring = 'f1')
        grid_search.fit(x_train, y_train)
        print(grid_search.best_estimator_)
        print(grid_search.best_score_)

        y_predict = grid_search.predict(x_test)

        count_right = 0
        count_wrong = 0

        detected_cancer = 0
        detected_wrong = 0
        total_cancer = 0
        for i in range((y_predict.shape)[0]):
            if y_test[i] == 1:
                total_cancer = total_cancer + 1


            if y_predict[i] == y_test[i]:
                if y_test[i] != 0:
                    detected_cancer = detected_cancer + 1
                count_right = count_right + 1
            else:
                if y_test[i] == 0:
                    detected_wrong = detected_wrong + 1
                count_wrong = count_wrong + 1
        
        print("correction rate: "+str(count_right / (count_wrong + count_right)) + ". Successfully detected cancer: "+str(detected_cancer)
        +". Cancer detection rate: "+str(detected_cancer/total_cancer)+". Wrong diagnosis: "+str(detected_wrong),". Total cancer: "+str(total_cancer))



fold 0
KNeighborsClassifier(n_neighbors=1, p=3, weights='distance')
0.12753929687587365
correction rate: 0.7582655826558266. Successfully detected cancer: 43. Cancer detection rate: 0.14381270903010032. Wrong diagnosis: 190 . Total cancer: 299
fold 1
KNeighborsClassifier(n_neighbors=1, p=1, weights='distance')
0.15425176444850058
correction rate: 0.7859078590785907. Successfully detected cancer: 35. Cancer detection rate: 0.13108614232209737. Wrong diagnosis: 163 . Total cancer: 267
fold 2
KNeighborsClassifier(n_neighbors=1, p=1, weights='distance')
0.16438234188935047
correction rate: 0.7804878048780488. Successfully detected cancer: 38. Cancer detection rate: 0.14339622641509434. Wrong diagnosis: 178 . Total cancer: 265
fold 3


KeyboardInterrupt: 

In [ ]:

    # ============================================
    def predict_Knn(test_vec, train_data, train_label, k):
        res_list = []
        dis_list = []
        train_data = train_data.astype(float)
        test_vec = test_vec.astype(float)
        for i in range(train_data.shape[1]):
            diff = (test_vec.reshape((train_data.shape[0],1))) - train_data[:,i]
            dis_list.append(np.linalg.norm(diff))

            
        # Smallest K elements indices
        # using sorted() + lambda + list slicing
        res = sorted(range(len(dis_list)), key = lambda sub: dis_list[sub])[:k] 

        for i in range(len(res)):
            res_list.append(train_label[res[i]])

        count_0 = 0
        count_1 = 0

        for i in range(len(res_list)):
            if res_list[i] == 0:
                count_0 = count_0 + 1
            else:
                count_1 = count_1 + 1

        #if count_1 > count_0:
        #    return 1
        #else:
        #    return 0        

        # as long as one nearest neighbour has cancer,
        # than we predict cancer
        if count_1 > 0:
            return 1
        else:
            return 0